In [ ]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_excel('retail_store_inventory.xlsx')

# Convert 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Filter data based on year 2023
filtered_df = df[df['Date'].dt.year == 2022 & 2023]

# Drop rows with missing values in critical columns
filtered_df = filtered_df.dropna(subset=['Product ID', 'Store ID', 'Inventory Level', 'Units Sold'])

# Extract relevant columns for optimization
relevant_columns = ['Store ID', 'Product ID', 'Inventory Level', 'Units Sold', 'Units Ordered']

# Subset the dataset
data = filtered_df[relevant_columns].copy()

# Optionally, aggregate data by product id and store id if needed
df_grouped = data.groupby(['Store ID', 'Product ID']).agg({
    'Inventory Level': 'mean', 
    'Units Sold': 'mean', 
    'Units Ordered': 'mean', 
}).reset_index()

# Remove decimals for the relevant columns (convert to integers)
df_grouped['Inventory Level'] = df_grouped['Inventory Level'].round().astype(int)  # Round and convert to int
df_grouped['Units Sold'] = df_grouped['Units Sold'].astype(int)  # Convert to int
df_grouped['Units Ordered'] = df_grouped['Units Ordered'].astype(int)  # Convert to int
df_grouped['Demand Forecast'] = df_grouped['Demand Forecast'].round().astype(int)  # Round and convert to int
df_grouped['Price'] = df_grouped['Price'].round(2)  # Price might still need decimals, so rounding to 2 decimals


df_grouped.head()



,Store ID,Product ID,Inventory Level,Units Sold,Units Ordered,Demand Forecast,Price
0,S001,P0001,271,138,111,13051,5096.96
1,S001,P0002,278,144,113,13315,5020.77
2,S001,P0003,269,138,112,13092,4855.62
3,S001,P0004,284,139,108,12993,4881.72
4,S001,P0005,271,133,105,12192,5036.04


In [ ]:
# Constants (you need to define these based on your business model)
HOLDING_COST_PER_UNIT = 1   # Cost to hold one unit of inventory
STOCKOUT_COST_PER_UNIT = 5  # Cost for a stockout event
COST_PER_UNIT_ORDERED = 2  # Cost to order one unit of inventory
FIXED_ORDERING_COST = 50   # Fixed cost for placing an order

def calculate_total_cost(store_id, product_id, inventory_level, units_sold, demand_forecast, restock_quantity):
    """
    Calculate the total cost for a store and product combination.

    Parameters:
    - store_id: ID of the store
    - product_id: ID of the product
    - inventory_level: Current inventory level
    - units_sold: Units sold (used for stockout calculation)
    - demand_forecast: Forecasted demand for the product
    - restock_quantity: Number of units to order for restocking

    Returns:
    - Total cost: The calculated total cost (holding + stockout + ordering)
    """

    # Holding Cost: Cost of keeping inventory in stock
    holding_cost = inventory_level * HOLDING_COST_PER_UNIT

    # Stockout Cost: Cost of missing sales due to insufficient inventory
    stockout_cost = max(0, demand_forecast - inventory_level) * STOCKOUT_COST_PER_UNIT

    # Ordering Cost: Cost of placing orders to restock
    ordering_cost = restock_quantity * COST_PER_UNIT_ORDERED + FIXED_ORDERING_COST

    # Total Cost
    total_cost = holding_cost + stockout_cost + ordering_cost

    return total_cost


In [ ]:
# Before optimization: assuming a simple heuristic or current data
def calculate_initial_costs(df_grouped):
    costs_before = []
    
    for index, row in df_grouped.iterrows():
        store_id = row['Store ID']
        product_id = row['Product ID']
        inventory_level = row['Inventory Level']
        units_sold = row['Units Sold']
        demand_forecast = row['Demand Forecast']
        
        # For initial restocking, assume restock quantity is the difference between demand and inventory level
        restock_quantity = max(0, demand_forecast - inventory_level)  # Basic heuristic: restock to match demand
        
        # Calculate the cost before optimization
        cost_before = calculate_total_cost(store_id, product_id, inventory_level, units_sold, demand_forecast, restock_quantity)
        costs_before.append(cost_before)
    
    df_grouped['Cost Before'] = costs_before
    return df_grouped

# Apply the initial cost calculation
df_grouped = calculate_initial_costs(df_grouped)
df_grouped.head()


In [ ]:
def objective_function(store_id, product_id, inventory_level, units_sold, demand_forecast, restock_threshold, restock_quantity):
    """
    This objective function will be used for Tabu Search. The goal is to minimize the total cost.
    """

    # Adjust inventory level based on restock threshold (simulate restocking)
    if inventory_level < restock_threshold:
        inventory_level += restock_quantity  # Restock to the threshold

    # Calculate the total cost with the current restocking strategy
    total_cost = calculate_total_cost(store_id, product_id, inventory_level, units_sold, demand_forecast, restock_quantity)

    return total_cost
